In [ ]:
# Installer deep-translator si nécessaire
!pip install deep-translator > /dev/null 2>&1

# Utilisation de deep-translator pour effectuer une traduction
from deep_translator import GoogleTranslator

# Exemple de traduction
translator = GoogleTranslator(source='auto', target='en')
translation = translator.translate('Bonjour tout le monde!')
print(f"Traduction : {translation}")

# Vérifier la version de deep-translator
import deep_translator
print("deep-translator version:", deep_translator.__version__)

# Installer d'autres dépendances nécessaires
!pip install biopython pandas openpyxl matplotlib > /dev/null 2>&1
!pip install sqlalchemy psycopg2-binary > /dev/null 2>&1
!pip install wordcloud > /dev/null 2>&1

# Importation des bibliothèques
from Bio import Entrez
import pandas as pd
import openai
import matplotlib.pyplot as plt
from wordcloud import WordCloud



# Configuration de l'API PubMed
Entrez.email = "worthylastchance@gmail.com"
openai.api_key = "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"


# Fonction pour rechercher des articles PubMed en fonction d'une requête
def search_pubmed(query, max_results=5):
    handle = Entrez.esearch(db="pubmed", term=query, retmax=max_results)
    record = Entrez.read(handle)
    handle.close()
    return record["IdList"]

def fetch_article_details(pubmed_id):
    handle = Entrez.efetch(db="pubmed", id=pubmed_id, rettype="xml")
    record = Entrez.read(handle)
    handle.close()

    article = record['PubmedArticle'][0]['MedlineCitation']['Article']
    title = article['ArticleTitle']
    abstract = article['Abstract']['AbstractText'][0] if 'Abstract' in article else "Résumé non disponible"
    journal = article['Journal']['Title']
    pub_date = article['Journal']['JournalIssue']['PubDate'].get('Year', 'Date inconnue')
    doi = article.get('ELocationID', ['DOI non disponible'])[0]
    # Modification de la ligne pour vérifier si les clés existent
    authors = ", ".join([f"{author.get('ForeName', '')} {author.get('LastName', '')}".strip()
                     for author in article.get("AuthorList", [])]) if "AuthorList" in article else "Auteurs non disponibles"


    # Extraction correcte des mots-clés
    keyword_list = record['PubmedArticle'][0]['MedlineCitation'].get('KeywordList', [])
    keywords = ", ".join([str(keyword) for keyword in keyword_list])

    return {
        "ID": pubmed_id,
        "Title": title,
        "Abstract": abstract,
        "Journal": journal,
        "PublicationDate": pub_date,
        "DOI": doi,
        "Authors": authors,
        "Keywords": keywords
    }

    return {
        "ID": pubmed_id,
        "Title": title,
        "Abstract": abstract,
        "Journal": journal,
        "PublicationDate": pub_date,
        "DOI": doi,
        "Authors": authors,
        "Keywords": keywords
    }

# Fonction pour générer un rapport d'article
def generate_report(article, language="en"):
    print("\n--- Rapport d'Article ---")
    print(f"ID : {article['ID']}")
    print(f"Titre : {article['Title']}")
    print(f"Résumé (original) : {article['Abstract']}")
    print(f"Journal : {article['Journal']}")
    print(f"Date de publication : {article['PublicationDate']}")
    print(f"Auteurs : {article['Authors']}")
    print(f"Mots-clés : {article['Keywords']}")
    print(f"DOI : {article['DOI']}")

    # Résumé traduit
    translated_summary = translator.translate(article['Abstract'], dest=language)

    # Si le résultat est une chaîne (str), l'assigner directement
    if isinstance(translated_summary, str):
        translated_summary = translated_summary
    else:
        translated_summary = translated_summary.text

    print(f"\nRésumé traduit ({language}) : {translated_summary}")
    print("--------------------------")


# Fonction principale pour le chatbot interactif
def chatbot_mode():
    print("Bienvenue dans le chatbot interactif pour PubMed.")
    saved_articles = []

    while True:
        print("\nQue voulez-vous faire ?")
        print("1. Rechercher des articles")
        print("2. Résumer un article avec un ID spécifique (dans une langue choisie)")
        print("3. Afficher les détails d'un article avec un ID spécifique")
        print("4. Quitter")

        choice = input("Entrez votre choix (1, 2, 3 ou 4) : ")

        if choice == "1":
            query = input("\nEntrez votre requête de recherche : ")
            print("\nRecherche en cours...")
            pubmed_ids = search_pubmed(query, max_results=5)

            if not pubmed_ids:
                print("Aucun article trouvé pour cette requête.")
                continue

            articles = [fetch_article_details(pubmed_id) for pubmed_id in pubmed_ids]
            saved_articles.extend(articles)

            # Sauvegarde dans un fichier CSV
            file_path = "pubmed_articles.csv"
            pd.DataFrame(saved_articles).to_csv(file_path, index=False)
            print(f"\nLes articles ont été sauvegardés dans le fichier : {file_path}")

            for article in articles:
                print("\n------------------------------------------")
                print(f"ID : {article['ID']}")
                print(f"Titre : {article['Title']}")
                print(f"Résumé : {article['Abstract']}")
                print(f"Journal : {article['Journal']}")
                print(f"Date de publication : {article['PublicationDate']}")
                print(f"DOI : {article['DOI']}")
                print("------------------------------------------")

        elif choice == "2":
            if not saved_articles:
                print("Aucun article sauvegardé. Effectuez une recherche d'abord.")
                continue

            article_id = input("Entrez l'ID de l'article à résumer : ")
            language = input("Entrez la langue souhaitée (par ex., 'fr' pour français, 'en' pour anglais) : ").strip().lower()
            article = next((art for art in saved_articles if art["ID"] == article_id), None)
            if article:
                generate_report(article, language)
            else:
                print(f"Aucun article trouvé avec l'ID {article_id}.")

        elif choice == "3":
            if not saved_articles:
                print("Aucun article sauvegardé. Effectuez une recherche d'abord.")
                continue

            article_id = input("Entrez l'ID de l'article pour les détails : ")
            article = next((art for art in saved_articles if art["ID"] == article_id), None)
            if article:
                print("\n--- Détails supplémentaires de l'article ---")
                print(f"Titre : {article['Title']}")
                print(f"Auteurs : {article['Authors']}")
                print(f"Mots-clés : {article['Keywords']}")
                print(f"Résumé : {article['Abstract']}")
                print(f"Journal : {article['Journal']}")
                print(f"DOI : {article['DOI']}")
                print("------------------------------------------")
            else:
                print(f"Aucun article trouvé avec l'ID {article_id}.")

        elif choice == "4":
            print("Merci d'avoir utilisé le chatbot PubMed. À bientôt !")
            break

        else:
            print("Choix invalide. Veuillez réessayer.")

# Lancement du mode chatbot
if __name__ == "__main__":
    chatbot_mode()


Traduction : Hello everyone!
deep-translator version: 1.9.1
Bienvenue dans le chatbot interactif pour PubMed.

Que voulez-vous faire ?
1. Rechercher des articles
2. Résumer un article avec un ID spécifique (dans une langue choisie)
3. Afficher les détails d'un article avec un ID spécifique
4. Quitter
Entrez votre choix (1, 2, 3 ou 4) : 1

Entrez votre requête de recherche : soft mist inhalers copd asthma

Recherche en cours...

Les articles ont été sauvegardés dans le fichier : pubmed_articles.csv

------------------------------------------
ID : 39568351
Titre : [Sustainability and choice of inhaled treatment for obstructive pulmonary diseases].
Résumé : Inhaled treatments are essential for obstructive lung diseases such as asthma and chronic obstructive pulmonary disease (COPD). However, pressurized metered-dose inhalers (pMDI) use propellant gases, contributing to greenhouse gas (GHG) emissions and thereby to climate change. The use of dry powder inhalers (DPI) or soft mist inhalers 

In [2]:
# Installer les dépendances si nécessaire
!pip install fpdf deep-translator biopython pandas openpyxl matplotlib sqlalchemy psycopg2-binary wordcloud > /dev/null 2>&1

# Importation des bibliothèques nécessaires
from fpdf import FPDF
from deep_translator import GoogleTranslator
from Bio import Entrez
import pandas as pd
import openai
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Configuration de l'API PubMed
Entrez.email = "worthylastchance@gmail.com"
openai.api_key = "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"

# Initialisation du traducteur
translator = GoogleTranslator(source='auto', target='en')

# Fonction pour rechercher des articles PubMed en fonction d'une requête
def search_pubmed(query, max_results=5):
    handle = Entrez.esearch(db="pubmed", term=query, retmax=max_results)
    record = Entrez.read(handle)
    handle.close()
    return record["IdList"]

# Fonction pour récupérer les détails d'un article PubMed
def fetch_article_details(pubmed_id):
    handle = Entrez.efetch(db="pubmed", id=pubmed_id, rettype="xml")
    record = Entrez.read(handle)
    handle.close()

    article = record['PubmedArticle'][0]['MedlineCitation']['Article']
    title = article['ArticleTitle']
    abstract = article['Abstract']['AbstractText'][0] if 'Abstract' in article else "Résumé non disponible"
    journal = article['Journal']['Title']
    pub_date = article['Journal']['JournalIssue']['PubDate'].get('Year', 'Date inconnue')
    doi = article.get('ELocationID', ['DOI non disponible'])[0]
    authors = ", ".join([f"{author.get('ForeName', '')} {author.get('LastName', '')}".strip()
                         for author in article.get("AuthorList", [])]) if "AuthorList" in article else "Auteurs non disponibles"

    # Extraction correcte des mots-clés
    keyword_list = record['PubmedArticle'][0]['MedlineCitation'].get('KeywordList', [])
    keywords = ", ".join([str(keyword) for keyword in keyword_list])

    return {
        "ID": pubmed_id,
        "Title": title,
        "Abstract": abstract,
        "Journal": journal,
        "PublicationDate": pub_date,
        "DOI": doi,
        "Authors": authors,
        "Keywords": keywords
    }

# Fonction pour générer un rapport d'article
def generate_report(article, language="en"):
    print("\n--- Rapport d'Article ---")
    print(f"ID : {article['ID']}")
    print(f"Titre : {article['Title']}")
    print(f"Résumé (original) : {article['Abstract']}")
    print(f"Journal : {article['Journal']}")
    print(f"Date de publication : {article['PublicationDate']}")
    print(f"Auteurs : {article['Authors']}")
    print(f"Mots-clés : {article['Keywords']}")
    print(f"DOI : {article['DOI']}")

    # Résumé traduit
    translated_summary = translator.translate(article['Abstract'], dest=language)

    # Si le résultat est une chaîne (str), l'assigner directement
    if isinstance(translated_summary, str):
        translated_summary = translated_summary
    else:
        translated_summary = translated_summary.text

    print(f"\nRésumé traduit ({language}) : {translated_summary}")
    print("--------------------------")

# Fonction pour sauvegarder le chat entier en format PDF
!pip install reportlab
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

def save_chat_to_pdf(chat_messages, filename="chat_history.pdf"):
    """
    Enregistre l'historique du chat dans un fichier PDF.

    Args:
        chat_messages (list): Liste des messages du chat.
        filename (str): Nom du fichier PDF à générer.
    """
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter
    y_position = height - 40  # Position initiale en haut de la page
    line_height = 15          # Hauteur entre les lignes

    c.setFont("Helvetica", 10)

    for message in chat_messages:
        if y_position < 40:  # Nouvelle page si l'espace est insuffisant
            c.showPage()
            c.setFont("Helvetica", 10)
            y_position = height - 40

        c.drawString(40, y_position, message)
        y_position -= line_height

    c.save()
    print(f"Le chat a été sauvegardé dans le fichier {filename}")


# Fonction principale du chatbot
def chatbot_mode():
    print("Bienvenue dans le chatbot interactif pour PubMed.")
    saved_articles = []
    chat_messages = []  # Liste pour stocker les messages du chat

    while True:
        print("\nQue voulez-vous faire ?")
        print("1. Rechercher des articles")
        print("2. Résumer un article avec un ID spécifique (dans une langue choisie)")
        print("3. Afficher les détails d'un article avec un ID spécifique")
        print("4. Télécharger le chat entier en PDF")
        print("5. Quitter")

        choice = input("Entrez votre choix (1, 2, 3, 4 ou 5) : ")

        if choice == "1":
            query = input("\nEntrez votre requête de recherche : ")
            chat_messages.append(f"Recherche: {query}")
            print("\nRecherche en cours...")
            pubmed_ids = search_pubmed(query, max_results=5)

            if not pubmed_ids:
                print("Aucun article trouvé pour cette requête.")
                continue

            articles = [fetch_article_details(pubmed_id) for pubmed_id in pubmed_ids]
            saved_articles.extend(articles)

            # Sauvegarde dans un fichier CSV
            file_path = "pubmed_articles.csv"
            pd.DataFrame(saved_articles).to_csv(file_path, index=False)
            chat_messages.append(f"Les articles ont été sauvegardés dans le fichier : {file_path}")
            print(f"\nLes articles ont été sauvegardés dans le fichier : {file_path}")

            for article in articles:
                print("\n------------------------------------------")
                print(f"ID : {article['ID']}")
                print(f"Titre : {article['Title']}")
                print(f"Résumé : {article['Abstract']}")
                print(f"Journal : {article['Journal']}")
                print(f"Date de publication : {article['PublicationDate']}")
                print(f"DOI : {article['DOI']}")
                print("------------------------------------------")

        elif choice == "2":
            if not saved_articles:
                print("Aucun article sauvegardé. Effectuez une recherche d'abord.")
                continue

            article_id = input("Entrez l'ID de l'article à résumer : ")
            language = input("Entrez la langue souhaitée (par ex., 'fr' pour français, 'en' pour anglais) : ").strip().lower()
            article = next((art for art in saved_articles if art["ID"] == article_id), None)
            if article:
                generate_report(article, language)
                chat_messages.append(f"Résumé de l'article ID {article_id} dans la langue {language}")
            else:
                print(f"Aucun article trouvé avec l'ID {article_id}.")

        elif choice == "3":
            if not saved_articles:
                print("Aucun article sauvegardé. Effectuez une recherche d'abord.")
                continue

            article_id = input("Entrez l'ID de l'article pour les détails : ")
            article = next((art for art in saved_articles if art["ID"] == article_id), None)
            if article:
                print("\n--- Détails supplémentaires de l'article ---")
                print(f"Titre : {article['Title']}")
                print(f"Auteurs : {article['Authors']}")
                print(f"Mots-clés : {article['Keywords']}")
                print(f"Résumé : {article['Abstract']}")
                print(f"Journal : {article['Journal']}")
                print(f"DOI : {article['DOI']}")
                print("------------------------------------------")
                chat_messages.append(f"Détails de l'article ID {article_id}")
            else:
                print(f"Aucun article trouvé avec l'ID {article_id}.")

        elif choice == "4":
              save_chat_to_pdf(chat_messages)
              chat_messages.append("Chat téléchargé en PDF.")


        elif choice == "5":
            print("Merci d'avoir utilisé le chatbot PubMed. À bientôt !")
            break

        else:
            print("Choix invalide. Veuillez réessayer.")

# Lancement du mode chatbot
if __name__ == "__main__":
    chatbot_mode()



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.1 MB/s eta 0:00:00
Bienvenue dans le chatbot interactif pour PubMed.

Que voulez-vous faire ?
1. Rechercher des articles
2. Résumer un article avec un ID spécifique (dans une langue choisie)
3. Afficher les détails d'un article avec un ID spécifique
4. Télécharger le chat entier en PDF
5. Quitter
Entrez votre choix (1, 2, 3, 4 ou 5) : 1

Entrez votre requête de recherche : soft mist inhalers

Recherche en cours...

Les articles ont été sauvegardés dans le fichier : pubmed_articles.csv

------------------------------------------
ID : 39638699
Titre : Environmental impact of inhaler therapies in a cystic fibrosis unit: Strategies for sustainability.
Résumé : Inhaled therapy is essential in cystic fibrosis; however, inhalers have a significant environmental impact due to the greenhouse gases (GHGs) emitted. The environmental impact of a product is estimated by its carbon footprint (CF). Pressurized metered-dose inhalers (pMDIs) hav

In [8]:
# Installer les dépendances si nécessaire
!pip install fpdf deep-translator biopython pandas openpyxl matplotlib sqlalchemy psycopg2-binary wordcloud reportlab > /dev/null 2>&1

# Importation des bibliothèques nécessaires
from fpdf import FPDF
from deep_translator import GoogleTranslator
from Bio import Entrez
import pandas as pd
import openai
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from reportlab.lib.pagesizes import letter
from reportlab.platypus import Paragraph, SimpleDocTemplate, Spacer
from reportlab.lib.styles import getSampleStyleSheet

from fpdf import FPDF
import pandas as pd

# Configuration de l'API PubMed
Entrez.email = "worthylastchance@gmail.com"
openai.api_key = "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"

# Initialisation du traducteur
translator = GoogleTranslator(source='auto', target='en')
# Classe PDF pour gérer la mise en forme
class ChatPDF(FPDF):
    def header(self):
        self.set_font("Arial", size=12)
        self.cell(0, 10, "Chatbot Interactif pour PubMed", ln=True, align="C")
        self.ln(5)

    def footer(self):
        self.set_y(-15)
        self.set_font("Arial", size=8)
        self.cell(0, 10, f'Page {self.page_no()}', align="C")

# Fonction pour sauvegarder le chat en PDF
def save_chat_to_pdf(chat_messages, pdf_file="Chatbot_Interactions.pdf"):
    """
    Enregistre l'historique du chat dans un fichier PDF.
    """
    pdf = ChatPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    # Ajout de chaque message dans le PDF
    for message in chat_messages:
        if "Utilisateur" in message:
            pdf.set_font("Arial", "B", 12)
        elif "Bot" in message:
            pdf.set_font("Arial", "I", 12)
        else:
            pdf.set_font("Arial", size=12)
        pdf.multi_cell(0, 10, message)
        pdf.ln(5)

    # Sauvegarde du fichier PDF
    pdf.output(pdf_file)
    print(f"Chat sauvegardé dans le fichier PDF : {pdf_file}")

# Fonction pour rechercher des articles sur PubMed (placeholder)
def search_pubmed(query, max_results=5):
    """
    Recherche des articles sur PubMed en fonction de la requête.
    Args:
        query (str): La requête de recherche.
        max_results (int): Nombre maximum de résultats.
    Returns:
        list: Liste des identifiants PubMed (PMID).
    """
    handle = Entrez.esearch(db="pubmed", term=query, retmax=max_results)
    record = Entrez.read(handle)
    handle.close()
    return record.get("IdList", [])


# Fonction pour récupérer les détails d'un article (placeholder)
def fetch_article_details(pubmed_id):
    """
    Récupère les détails d'un article PubMed en utilisant son identifiant.
    Args:
        pubmed_id (str): Identifiant PubMed (PMID).
    Returns:
        dict: Détails de l'article.
    """
    handle = Entrez.efetch(db="pubmed", id=pubmed_id, rettype="xml", retmode="text")
    record = Entrez.read(handle)
    handle.close()

    try:
        article = record['PubmedArticle'][0]['MedlineCitation']['Article']
        title = article.get('ArticleTitle', 'Titre non disponible')
        abstract = article.get('Abstract', {}).get('AbstractText', ['Résumé non disponible'])[0]
        journal = article.get('Journal', {}).get('Title', 'Journal non disponible')
        pub_date = article.get('Journal', {}).get('JournalIssue', {}).get('PubDate', {}).get('Year', 'Date inconnue')
        doi = article.get('ELocationID', ['DOI non disponible'])[0]
        authors = ", ".join([f"{a.get('ForeName', '')} {a.get('LastName', '')}".strip() for a in article.get("AuthorList", [])])

        return {
            "ID": pubmed_id,
            "Title": title,
            "Abstract": abstract,
            "Journal": journal,
            "PublicationDate": pub_date,
            "DOI": doi,
            "Authors": authors
        }
    except Exception as e:
        print(f"Erreur lors de la récupération des détails pour PMID {pubmed_id} : {e}")
        return {}

# Fonction pour sauvegarder les articles dans un fichier CSV
def save_articles_to_csv(articles, csv_file="pubmed_articles.csv"):
    """
    Sauvegarde les articles dans un fichier CSV.
    """
    df = pd.DataFrame(articles)
    df.to_csv(csv_file, index=False)
    print(f"Les articles ont été sauvegardés dans le fichier CSV : {csv_file}")

# Fonction principale du chatbot
def chatbot_mode():
    print("Bienvenue dans le chatbot interactif pour PubMed.")
    saved_articles = []
    chat_messages = []  # Liste pour stocker les messages du chat

    while True:
        print("\nQue voulez-vous faire ?")
        print("1. Rechercher des articles")
        print("2. Résumer un article avec un ID spécifique (dans une langue choisie)")
        print("3. Afficher les détails d'un article avec un ID spécifique")
        print("4. Télécharger le chat entier en PDF")
        print("5. Quitter")

        choice = input("Entrez votre choix (1, 2, 3, 4 ou 5) : ")

        if choice == "1":
            query = input("\nEntrez votre requête de recherche : ")
            chat_messages.append(f"Utilisateur : Recherche d'articles pour la requête '{query}'")
            print("\nRecherche en cours...")
            pubmed_ids = search_pubmed(query, max_results=5)

            if not pubmed_ids:
                print("Aucun article trouvé pour cette requête.")
                chat_messages.append(f"Bot : Aucun article trouvé pour la requête '{query}'.")
                continue

            articles = [fetch_article_details(pubmed_id) for pubmed_id in pubmed_ids]
            saved_articles.extend(articles)

            # Sauvegarde dans un fichier CSV
            save_articles_to_csv(saved_articles)
            chat_messages.append(f"Bot : Les articles ont été sauvegardés dans le fichier : pubmed_articles.csv")

            # Affichage des articles dans le terminal
            for article in articles:
                article_details = (
                    f"\n------------------------------------------\n"
                    f"ID : {article['ID']}\n"
                    f"Titre : {article['Title']}\n"
                    f"Résumé : {article['Abstract']}\n"
                    f"Journal : {article['Journal']}\n"
                    f"Date de publication : {article['PublicationDate']}\n"
                    f"DOI : {article['DOI']}\n"
                    f"------------------------------------------"
                )
                print(article_details)
                chat_messages.append(f"Bot : {article_details}")

        elif choice == "2":
            if not saved_articles:
                print("Aucun article sauvegardé. Effectuez une recherche d'abord.")
                chat_messages.append("Bot : Aucun article sauvegardé. Effectuez une recherche d'abord.")
                continue

            article_id = input("Entrez l'ID de l'article à résumer : ")
            language = input("Entrez la langue souhaitée (par ex., 'fr' pour français, 'en' pour anglais) : ").strip().lower()
            article = next((art for art in saved_articles if art["ID"] == article_id), None)
            if article:
                print(f"Résumé de l'article {article_id} généré.")
                chat_messages.append(f"Utilisateur : Résumé demandé pour l'article {article_id} en {language}.")
                chat_messages.append(f"Bot : Résumé de l'article ID {article_id} dans la langue {language}.")
            else:
                print(f"Aucun article trouvé avec l'ID {article_id}.")
                chat_messages.append(f"Bot : Aucun article trouvé avec l'ID {article_id}.")

        elif choice == "3":
            if not saved_articles:
                print("Aucun article sauvegardé. Effectuez une recherche d'abord.")
                chat_messages.append("Bot : Aucun article sauvegardé. Effectuez une recherche d'abord.")
                continue

            article_id = input("Entrez l'ID de l'article pour les détails : ")
            article = next((art for art in saved_articles if art["ID"] == article_id), None)
            if article:
                details = (
                    f"\n--- Détails supplémentaires de l'article ---\n"
                    f"Titre : {article['Title']}\n"
                    f"Auteurs : {article.get('Authors', 'Non disponible')}\n"
                    f"Mots-clés : {article.get('Keywords', 'Non disponible')}\n"
                    f"Résumé : {article['Abstract']}\n"
                    f"Journal : {article['Journal']}\n"
                    f"DOI : {article['DOI']}\n"
                    f"------------------------------------------"
                )
                print(details)
                chat_messages.append(f"Bot : {details}")
            else:
                print(f"Aucun article trouvé avec l'ID {article_id}.")
                chat_messages.append(f"Bot : Aucun article trouvé avec l'ID {article_id}.")

        elif choice == "4":
            print("Téléchargement de l'historique du chat en cours...")
            save_chat_to_pdf(chat_messages)
            chat_messages.append("Utilisateur : Téléchargement du chat demandé.")
            chat_messages.append("Bot : Chat sauvegardé en PDF.")

        elif choice == "5":
            print("Merci d'avoir utilisé le chatbot PubMed. À bientôt !")
            chat_messages.append("Utilisateur : Quitter.")
            chat_messages.append("Bot : Merci d'avoir utilisé le chatbot PubMed. À bientôt !")
            break

        else:
            print("Choix invalide. Veuillez réessayer.")
            chat_messages.append("Bot : Choix invalide.")

# Exécution du chatbot
if __name__ == "__main__":
    chatbot_mode()


Bienvenue dans le chatbot interactif pour PubMed.

Que voulez-vous faire ?
1. Rechercher des articles
2. Résumer un article avec un ID spécifique (dans une langue choisie)
3. Afficher les détails d'un article avec un ID spécifique
4. Télécharger le chat entier en PDF
5. Quitter
Entrez votre choix (1, 2, 3, 4 ou 5) : 1

Entrez votre requête de recherche : soft mist inhalers

Recherche en cours...
Les articles ont été sauvegardés dans le fichier CSV : pubmed_articles.csv

------------------------------------------
ID : 39638699
Titre : Environmental impact of inhaler therapies in a cystic fibrosis unit: Strategies for sustainability.
Résumé : Inhaled therapy is essential in cystic fibrosis; however, inhalers have a significant environmental impact due to the greenhouse gases (GHGs) emitted. The environmental impact of a product is estimated by its carbon footprint (CF). Pressurized metered-dose inhalers (pMDIs) have a higher CF than dry powder inhalers (DPIs) and soft mist inhalers (SMIs

UnicodeEncodeError: 'latin-1' codec can't encode character '\u03b2' in position 1249: ordinal not in range(256)